In [6]:
from IPython.display import clear_output

# !pip install bs4

clear_output()
print('packages istalled')

packages istalled


In [19]:
ROOT_DATA_DIR = '/srv/data'

ROOT_DATA_DIR = '../../../Downloads/data'

def get_path(dir_name):
    root_data_dir = os.environ['ROOT_DATA_DIR']
    return os.path.join(root_data_dir, dir_name)

[semrush: leafly competitors](https://www.semrush.com/website/leafly.com/competitors/)

[Search results](https://www.allbud.com/marijuana-strains/search?results=40)



In [9]:
from bs4 import BeautifulSoup

def get_artists_pages(result_csv_path: str):
    page_list: List[str] = prepare_pages_list()
    logger.info(result_csv_path)
    
    if not os.path.exists(result_csv_path):
        res_df = pd.DataFrame([], columns=['artist_name', 'artist_link'])
        for current_alphabet_page in page_list:
            artists_scraper = BeautifulSoup(
                markup=requests.get(current_alphabet_page).content, features="html.parser"
            )
            res = artists_scraper.find(name="div", class_='masonry-text-view masonry-text-view-all')
            artists = res.find_all(name='li')

            res = []
            for artist in artists:
                artist_info = artist.find('a')
                res.append((artist_info['href'], artist_info.get_text()))
            current_page_df = pd.DataFrame(res, columns=['artist_name', 'artist_link'])
            res_df = pd.concat([res_df, current_page_df])
        res_df.to_csv(result_csv_path, index=False)
        logger.info("Artists pages saved to %s: %d rows", result_csv_path, res_df.shape[0])

In [10]:
import os

input_file_path = os.path.join(ROOT_DATA_DIR, 'AllBud.html')

with open(input_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()
items_scraper = BeautifulSoup(
    markup=html_content, features="html.parser"
)

In [11]:
import pandas as pd

import re

catalog_data_path = os.path.join(ROOT_DATA_DIR, 'allbud_catalog.csv')

if not os.path.exists(catalog_data_path):
    print('Extracting from HTML')
    df_entries = []
    items = items_scraper.find_all(name='div', class_="col-sm-6 col-md-4 col-lg-3")
    for i in items:
        title = i.find(class_='object-title').find(class_='visible-xs').text
        link = i.find(class_='object-title').find('a')['href']
        tags = '|'.join([t.strip() for t in i.find(class_='subtitle').text.split('\n') if len(t.strip()) > 1])
        percentages = re.sub(r'\s+', ' ', i.find(class_='details').find(class_='percentages').text.replace('\n', ' '))
        num_favorites = i.find(class_='details').find(class_='favorites').text.replace('\n', ' ')
        avg_rating = i.find(class_='details').find(class_='ratings').find(class_='rating-num').text
        num_ratings = i.find(class_='details').find(class_='ratings').find(class_='rating-votes').find(class_='product-rating-votes').text.strip()
        df_entries.append((title, link, num_favorites, avg_rating, num_ratings, percentages, tags))
    catalog_df = pd.DataFrame(df_entries, columns=['title','link','num_favorites','avg_rating','num_ratings','percentages','tags'])
    
    catalog_df.to_csv(catalog_data_path, index=False)
else:
    print('reading from dump')
    catalog_df = pd.read_csv(catalog_data_path)
catalog_df.head()

reading from dump


,title,link,num_favorites,avg_rating,num_ratings,percentages,tags
0,Casey Jones,/marijuana-strains/sativa-dominant-hybrid/case...,1249 Favorites,4.6,38,THC: 22%,Sativa Dominant|80% Sativa/20% Indica
1,Gorilla Glue #4,/marijuana-strains/indica-dominant-hybrid/gori...,17105 Favorites,4.6,339,THC: 32%,Indica Dominant|60% Indica/40% Sativa
2,Cherry Pie,/marijuana-strains/indica-dominant-hybrid/cher...,3768 Favorites,4.5,110,"THC: 16% - 24%, CBD: 1%",Indica Dominant|80% Indica/20% Sativa
3,Amnesia Haze,/marijuana-strains/sativa-dominant-hybrid/amne...,2406 Favorites,4.5,55,"THC: 20% - 22%, CBD: 1%",Sativa Dominant|80% Sativa/20% Indica
4,Blueberry,/marijuana-strains/indica-dominant-hybrid/blue...,2880 Favorites,4.5,95,"THC: 16% - 24%, CBN: 1%",Indica Dominant|80% Indica/20% Sativa


In [34]:
def parse_tags(x):
    dominant, shares_str = x.split('|')
    sativa_share, indica_share = shares_str.split('/')
    return (dominant, sativa_share, indica_share)

catalog_df['tags'].head().apply(lambda x: parse_tags(x))

0    (Sativa Dominant, 80% Sativa, 20% Indica)
1    (Indica Dominant, 60% Indica, 40% Sativa)
2    (Indica Dominant, 80% Indica, 20% Sativa)
3    (Sativa Dominant, 80% Sativa, 20% Indica)
4    (Indica Dominant, 80% Indica, 20% Sativa)
Name: tags, dtype: object

In [21]:
import requests

def save_html(url, file_path):
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text
        
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(html_content)
    else:
        print("Failed to retrieve webpage %s. Status code:" % (url, response.status_code))

# root_data_dir = '/srv/data'
root_data_dir = get_path('pipelines-data')
html_data_dir = 'allbuds_html_pages'

html_dir = os.path.join(root_data_dir, html_data_dir)
if not os.path.exists(html_dir):
    os.mkdir(html_dir)
if len(os.listdir(html_dir)) == 0:
    for ind, row in catalog_df.iterrows():
        url = f"https://www.allbud.com{row['link']}"
        html_path = os.path.join(html_dir, f"{row['link'].replace('/', '_')}.html")
        if not os.path.exists(html_path):
            save_html(url, html_path)
        else:
            pass
        if ind % 50 == 0:
            print('%d out of %d' % (ind, catalog_df.shape[0]))
    clear_output()
print('All done!')

All done!


In [46]:
import json
import os


descriptions_json_path = os.path.join(root_data_dir, 'item_cards.json')
reviews_json_path = os.path.join(root_data_dir, 'reviews.json')

descrs = []
reviews = []
errors = []
if not os.path.exists(descriptions_json_path):
    for i in os.listdir(html_dir):
        if len(descrs) % 500 == 0:
            print('Processed: %s' % len(descrs))
        description_dict = {}
        input_file_path = os.path.join(html_dir, i)
        with open(input_file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        item_card_scraper = BeautifulSoup(
            markup=html_content, features="html.parser"
        )
        description_dict.update({
            "description": re.sub(r'\s+', ' ', item_card_scraper.find(class_='panel-body well description').text.replace('\n', ' '))
        })
        # details
        positive_effects = re.sub(r'\s+', ' ', item_card_scraper.find(name='section', id='positive-effects-mobile').text.replace('\n', ''))
        relieved_effects = re.sub(r'\s+', ' ', item_card_scraper.find(name='section', id='relieved-mobile').text.replace('\n', ''))
        flavours = re.sub(r'\s+', ' ', item_card_scraper.find(name='section', id='flavors-mobile').text.replace('\n', ''))
        aromas = re.sub(r'\s+', ' ', item_card_scraper.find(name='section', id='aromas-mobile').text.replace('\n', ''))
        description_dict.update({
            'item': i,
            'positive_effects': positive_effects,
            'relieved_effects': relieved_effects,
            'flavours': flavours,
            'aromas': aromas,
        })
        descrs.append(description_dict)
        revs = item_card_scraper.find(name='div', id='collapse_reviews')
        try:
            for r in revs.find_all(class_='infopanel review mobile-panel'):
                reviews.append({
                    'item': i,
                    'review': re.sub(r'\s+', ' ', r.find(class_='text').text.replace('\n', ''))
                })
        except AttributeError:
            errors.append(i)
    with open(descriptions_json_path, 'w') as json_file:
        json.dump(descrs, json_file)
    with open(reviews_json_path, 'w') as json_file:
        json.dump(reviews, json_file)
else:
    with open(descriptions_json_path, 'r') as f:
        print(f'Loading from {descriptions_json_path}..')
        descriptions_dict = json.load(f)
        print(list(descriptions_dict[0].keys()))
    with open(reviews_json_path, 'r') as json_file:
        print(f'Loading from {reviews_json_path}..')
        reviews_dict = json.load(json_file)
        print(list(reviews_dict[0].keys()))
print('All done, num descriptions %d, num reviews %d' % (len(descriptions_dict), len(reviews_dict)))
print('Num errors %d' % len(errors))

Loading from /Users/username/PycharmProjects/leaf-bro-data/data/pipelines-data/item_cards.json..
['description', 'item', 'positive_effects', 'relieved_effects', 'flavours', 'aromas']
Loading from /Users/username/PycharmProjects/leaf-bro-data/data/pipelines-data/reviews.json..
['item', 'review']
All done, num descriptions 4181, num reviews 13743
Num errors 0


In [50]:
import gzip 

def save_json_gzip(input_path, output_path):
    with open(input_path, 'r') as f_in:
        with gzip.open(output_path, 'wt', encoding='UTF-8') as f_out:
            data = json.load(f_in)
            json.dump(data, f_out)
    print(f'Saved to {output_path}')

def load_json_gzip(input_path):
    with gzip.open(input_path, 'rt', encoding='UTF-8') as f_in:
            data = json.load(f_in)
    print(f'Saved to {output_path}')
    return data

output_path = os.path.join(root_data_dir, 'item_cards.gzip')
# save_json_gzip(descriptions_json_path, output_path)
data = load_json_gzip(output_path)
print(len(data))

Saved to /Users/username/PycharmProjects/leaf-bro-data/data/pipelines-data/item_cards.gzip
4181


In [28]:
# for i in [descriptions_json_path, reviews_json_path]:
#     os.remove(i)

In [39]:
print(descriptions_dict[150]["description"])
print('---')
item_name = descriptions_dict[150]["item"]

 Sativa Dominant Hybrid - 65% Sativa / 35% IndicaTHC: 11%, CBD: 11% Blue Dream CBD is a slightly sativa dominant hybrid strain (65% sativa/35% indica) created through crossing the powerful Blue Dream X California Orange CBD strains. Best known for its 1:1 CBD:THC ratio, Blue Dream CBD packs a dreamy high that will have you feeling kicked back and relaxed in no time at all. The high starts with a subtle build of effects, sneaking into the brain with a soothing and lifting feeling that pushes out any negative or racing thoughts and moods instantly. You'll find yourself filled with a light touch of focus and a slight boost in energy that lends itself well to jumping on your to-do list. A relaxing body high accompanies this happy lift, easing away aches and pains without causing any sedation. In combination with its low 11% THC level and super high 11% CBD level, these effects make Blue Dream CBD a great choice for treating a variety of conditions including inflammation, chronic pain, depr

In [41]:
item_name = '_marijuana-strains_indica_twilight-indica.html'
[i["review"] for i in reviews_dict if i['item']==item_name]

[' Let’s talk about this Indy Landrace…So many hybrids give you that nice variety in the effect and feeling. But there’s nothing like a beautiful pure Indy to help you drift off to sleep and make your feel better living in your body. This hits like that dank that you tried once as a kid and wish you could find again! Classic Afghan purity! ',
 ' Twilight is one of the best strains for my Chronic Migraines and PTSD. It also helps me relax and fall to sleep faster. I really wish that someone near me had this Great Strain! ',
 ' This strain showed off what it means to be a pure indica, for me. If you have issues with sleeping or just being still, this strain is perfect. Also, it helped with the muscle twitches I get in my upper quads. ',
 ' Real piney with sweet flower aroma. Sweet pine taste. Real kiefy light buds not dense. Over all a good flower to try close ro a Mazar sharif strain. ']

In [ ]:
import pandas as pd